In [1]:
import os
import numpy as np
import cv2
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sps
import math
from functools import reduce
from scipy.stats import bartlett
from scipy import stats
from scipy.stats import mannwhitneyu
import torch
from torchvision import datasets, models, transforms
import albumentations as A

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegressionCV

In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mpl

In [2]:
img_path_new = '/home/jupyter-fly_cv/Igor/Plants/data/Новый признак'

In [3]:
spatail_descr_df_2 = pd.read_csv('/home/jupyter-fly_cv/Igor/Plants/spatail_descr_2.csv')
spatail_descr_df = pd.read_csv('/home/jupyter-fly_cv/Igor/Plants/spatail_descr.csv')
annnot_df = pd.read_csv('/home/jupyter-fly_cv/Igor/Plants/annotation.csv',sep = ';')
#old_col_df = pd.read_csv('/home/jupyter-fly_cv/Igor/Plants/mean_color_and_hist_median.csv')

In [4]:
infer_aug = transforms.Compose([
        transforms.Resize(512),
       # transforms.CenterCrop(448),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [5]:
path_resnet_18_all = '/home/jupyter-fly_cv/Igor/Plants/ResNet18_all'
path_resnet_50_all = '/home/jupyter-fly_cv/Igor/Plants/ResNet50_all'
path_resnet_101_all = '/home/jupyter-fly_cv/Igor/Plants/ResNet101_all'

path_resnet_18_first = '/home/jupyter-fly_cv/Igor/Plants/ResNet18_first'
path_resnet_50_first = '/home/jupyter-fly_cv/Igor/Plants/ResNet50_first'
path_resnet_101_first = '/home/jupyter-fly_cv/Igor/Plants/ResNet101_first'

path_resnet_18_second = '/home/jupyter-fly_cv/Igor/Plants/ResNet18_second'
path_resnet_50_second = '/home/jupyter-fly_cv/Igor/Plants/ResNet50_second'
path_resnet_101_second = '/home/jupyter-fly_cv/Igor/Plants/ResNet101_second'

path_resnet_18_all_new = '/home/jupyter-fly_cv/Igor/Plants/ResNet18_all_2'
path_resnet_50_all_new = '/home/jupyter-fly_cv/Igor/Plants/ResNet50_all_2'
path_resnet_101_all_new = '/home/jupyter-fly_cv/Igor/Plants/ResNet101_all_2'

path_resnet_18_first_new = '/home/jupyter-fly_cv/Igor/Plants/ResNet18_first_2'
path_resnet_50_first_new = '/home/jupyter-fly_cv/Igor/Plants/ResNet50_first_2'
path_resnet_101_first_new = '/home/jupyter-fly_cv/Igor/Plants/ResNet101_first_2'

path_resnet_18_second_new = '/home/jupyter-fly_cv/Igor/Plants/ResNet18_second_2'
path_resnet_50_second_new = '/home/jupyter-fly_cv/Igor/Plants/ResNet50_second_2'
path_resnet_101_second_new = '/home/jupyter-fly_cv/Igor/Plants/ResNet101_second_2'

In [6]:
resnet_all = torch.load(path_resnet_50_all)
resnet_first = torch.load(path_resnet_18_first)
resnet_second = torch.load(path_resnet_18_second)

resnet_all_new = torch.load(path_resnet_50_all_new)
resnet_first_new = torch.load(path_resnet_50_first_new)
resnet_second_new = torch.load(path_resnet_18_second_new)

In [7]:
an_df = annnot_df.copy()
an_df.head()

,Порядковый номер,Номер растения,Устьица,хромосомы
0,1,Н16/1,гаплоид,гаплоид
1,2,Н16/2,диплоид,диплоид
2,3,991/1,гаплоид,гаплоид
3,4,991/2,диплоид,гаплоид
4,5,991/3,диплоид,диплоид


In [8]:
img_with_stick = ['454-80','454-33','453-4','454-60','454-32','449-1','991-3','454-24','452-5','451-12','454-68',
 '991-4','447-2','454-62','452-4','453-6','451-13','451-18','446-2','454-21','451-16','454-74','454-29','454-87',
 '991-6','451-19','453-2','446-4','453-8','454-26','454-84','452-3','454-77','447-1','451-4','454-19','452-9','991-2',
 '452-1','447-5','452-10','454-73','446-3','447-4','991-12','454-79','454-72','454-83','451-5','451-6','446-6','453-7',
 '454-78','454-75','454-30','454-63','991-1','454-66','453-3','454-49','991-9','454-71','454-59','454-67','454-56',
 '451-15','454-85','454-55','446-5','454-50','991-13','454-52','451-17','454-27','454-64','454-58','452-2','454-86',
 '454-54','454-82','454-76','451-11','454-20','454-28','454-31','454-22','454-23','454-57','454-25']

In [9]:
train = ['IMG_8409','IMG_8408','IMG_8407','IMG_8406','IMG_8405','IMG_8404',
        'IMG_8403','IMG_8402','IMG_8401','IMG_8400','IMG_8399','IMG_8398',
        'IMG_8397','IMG_8396','IMG_8395','IMG_8394','IMG_8393','IMG_8392',
        'IMG_8391','IMG_8390',
        'IMG_8440','IMG_8439','IMG_8438','IMG_8437','IMG_8436','IMG_8435',
        'IMG_8434','IMG_8433','IMG_8432','IMG_8431','IMG_8430','IMG_8410',
        'IMG_8344','IMG_8345','IMG_8346','IMG_8347','IMG_8348','IMG_8423',
        'IMG_8422','IMG_8421',
         
        'IMG_8428', 'IMG_8427','IMG_8426','IMG_8425','IMG_8424',
        'IMG_8339','IMG_8340','IMG_8341','IMG_8342','IMG_8349',
        'IMG_8329','IMG_8330',
        'IMG_8331','IMG_8332','IMG_8333','IMG_8334','IMG_8335','IMG_8336',
        'IMG_8337','IMG_8338',
        'IMG_8379','IMG_8378','IMG_8377','IMG_8376','IMG_8375','IMG_8374',
        'IMG_8373','IMG_8372','IMG_8371','IMG_8370','IMG_8369','IMG_8368',
        'IMG_8367', 'IMG_8366','IMG_8365','IMG_8364','IMG_8363','IMG_8362',
        'IMG_8361','IMG_8350']

val =['IMG_8389','IMG_8388','IMG_8387','IMG_8386','IMG_8385',
     'IMG_8420','IMG_8419','IMG_8418','IMG_8417','IMG_8416',#
     
      'IMG_8359','IMG_8358','IMG_8357','IMG_8356','IMG_8360', #
    'IMG_8324','IMG_8325','IMG_8326','IMG_8327','IMG_8328',]

test=['IMG_8384','IMG_8383','IMG_8382','IMG_8381','IMG_8380',
     'IMG_8415','IMG_8414','IMG_8413','IMG_8412','IMG_8411',
     
     'IMG_8319','IMG_8320','IMG_8321','IMG_8322','IMG_8323',
     'IMG_8354','IMG_8353','IMG_8352','IMG_8351','IMG_8355']

In [10]:
last = ['/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/1/2020_07_08/',
        '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/2/2020_10_08/',
        '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/',
        '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/4/2020_11_02/',
        '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/6/2020_11_18/',
       ]
first = ['/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/1/2020_06_26/',
         '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/2/2020_09_24/',
         '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_05/',
         '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/4/2020_10_20/',
         '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/6/2020_11_03/',
       ]

In [147]:
last_2 = ['/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/L-25/',
        '/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_24 Wheat/1102/',
       ]
first_2 = ['/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_11 Wheat/L-25/',
         '/home/jupyter-fly_cv/Igor/Plants/data/Новый признак/2022_03_11 Wheat/1102/',
       ]

In [12]:
def name_1_folder(path):
    names = path.split('/')
    for name in names:
        if name == '1':
            return True
    return False

In [13]:
def name_L_25_folder(path):
    names = path.split('/')
    for name in names:
        if name == 'L-25':
            return 1
    return 0

In [14]:
#black_list = ['446-4','454-30','451-11','451-17','454-20','454-22','454-28','454-83',
#             '454-55'] #1,2,2,2,2,2,2,2,1

In [15]:
# black_list = [#с цветокоррекцией
#             #не во всех временных точках
#             '454-85', '454-82', '454-78',
#              #1 991-13? 991-9? 454-71? 454-23? 451-11? 454-62? 454-56? 453-6? 451-10? 446-4?
#             '454-83','447-2','451-4','451-11','454-55','454-62','446-4',
#             '991-9','451-6','452-4','452-2','454-29','454-32','454-23','991-3',
#              #2 454-55? 454-25?  454-21? 991-4? 454-33? 454-27? 447-5?
#               '454-57','454-54','454-58','454-87','452-3','454-30','454-25',
#               '454-21','991-4','454-33', '454-27', '454-77'
#              ]

In [16]:
# black_list = [#без цветокоррекции
#             #не во всех временных точках
#             '454-85', '454-82', '454-78',
#              #1 '451-18' '454-75' '454-73' '454-71' '454-86' '446-6' '454-72','454-74','454-87','446-5'
#             '454-83','451-11','991-9','454-54','454-55','454-57','454-62',
#             '454-77','446-4','454-30','454-25','454-21','991-4','454-27',
#              #2 
             
#              #1 extra
#              '446-6','454-71','451-18','454-87','446-5', '454-86','446-6','454-72','454-73', '454-74',
#             '454-75','454-23','454-33', '451-4','454-32'
#             #2 extra
#             #'451-15','454-79','991-12'
#              ]

In [17]:
def refuse_null(name):
    names = name.split('-')
    if len(names) < 2:
        return name
    
    if names[1] == '01':
        names[1] = '1' 
    if names[1] == '02':
        names[1] = '2'
    if names[1] == '03':
        names[1] = '3'
    if names[1] == '04':
        names[1] = '4'
    if names[1] == '05':
        names[1] = '5'
    if names[1] == '06':
        names[1] = '6'
    if names[1] == '07':
        names[1] = '7'
    if names[1] == '08':
        names[1] = '8'
    if names[1] == '09':
        names[1] = '9'
    
    new_name = names[0]
    for n in names:
        if n == names[0]:
            continue
        new_name += '-' + n
    
    return new_name

In [18]:
def get_ploidn(tbl, name):
    name1 = name.split('-')[0]
    if len(name.split('-')) > 1:
        name2 = name.split('-')[1]
    else:
        name2 = ''
    for i in range(tbl.shape[0]):
        if name1 == tbl['Номер растения'].array[i].split('/')[0] and (name2 == tbl['Номер растения'].array[i].split('/')[1] or '0'+ name2 == tbl['Номер растения'].array[i].split('/')[1]):
                return tbl['Устьица'].array[i], tbl['хромосомы'].array[i]
    return '-', '-'  

In [19]:
def recover_annot_after_null(df):
    new_df = df.copy()
    for s in range(new_df.shape[0]):
        if  new_df.iloc[s].Хромосомы != '-' and new_df.iloc[s].Устьица != '-':
            continue
        name = new_df.iloc[s].Name
        name_df = df.query("Name == @name")
        ust = max(name_df['Устьица'])
        chrom = max(name_df['Хромосомы'])
        new_df.loc[s,'Устьица'] = ust
        new_df.loc[s, 'Хромосомы']= chrom
    return new_df

In [20]:
refuse_null('449-05-1')

'449-5-1'

In [21]:
spatail_descr_df_2.head()

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path
0,IMG_8319,1096.825469,2,6,3.0,8.5,28,519,801,40563,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
1,IMG_8346,1168.886435,5,10,5.0,12.0,38,720,637,38280,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
2,IMG_8345,933.494516,3,17,3.0,12.0,42,512,755,45812,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
3,IMG_8334,1294.366666,4,19,5.0,13.5,87,812,653,40995,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
4,IMG_8322,1108.915872,3,14,5.0,15.5,95,641,664,46825,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...


In [22]:
sp_df = spatail_descr_df.copy()
sp_df.Name = pd.Series(map(refuse_null,sp_df.Name))
sp_df = sp_df.assign(Устьица = pd.Series((map(lambda x: get_ploidn(an_df, x)[0],sp_df.Name))))
sp_df = sp_df.assign(Хромосомы = pd.Series((map(lambda x: get_ploidn(an_df, x)[0],sp_df.Name))))
sp_df = sp_df.assign(test = pd.Series((map(name_1_folder,sp_df.Path))))
sp_df = sp_df.query("test != True")
sp_df = sp_df.drop(columns = ['test'])
sp_df = sp_df.reset_index(drop=True)
sp_df = recover_annot_after_null(sp_df)
sp_df.head()

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Устьица,Хромосомы
0,453-4-1,1081.193001,8,39,8.0,18.5,40,528,941,92739,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,гаплоид,гаплоид
1,454-26-4,1358.567676,5,39,7.0,15.5,54,720,1146,117758,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,гаплоид,гаплоид
2,454-19,1438.165656,9,44,11.0,19.5,121,1159,582,91148,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,диплоид,диплоид
3,449-1-3,1380.912914,12,37,5.0,17.5,59,938,1078,87804,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,диплоид,диплоид
4,991-3-4,1222.470129,3,15,3.0,10.0,36,523,1148,50784,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,диплоид,диплоид


In [23]:
sp_2_df = spatail_descr_df_2.copy()
sp_2_df = sp_2_df.assign(L_25 = pd.Series((map(name_L_25_folder,sp_2_df.Path))))
sp_2_df.head()

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,L_25
0,IMG_8319,1096.825469,2,6,3.0,8.5,28,519,801,40563,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1
1,IMG_8346,1168.886435,5,10,5.0,12.0,38,720,637,38280,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1
2,IMG_8345,933.494516,3,17,3.0,12.0,42,512,755,45812,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1
3,IMG_8334,1294.366666,4,19,5.0,13.5,87,812,653,40995,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1
4,IMG_8322,1108.915872,3,14,5.0,15.5,95,641,664,46825,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1


In [24]:
df_2 = sp_2_df.copy()
df = sp_df.copy()

In [25]:
# b_m_df.query("Name == '446-2-1'")

In [26]:
# df = rec_df.merge(b_m_df)
# df.head()

In [27]:
# df['Median_H'] = col_df.Median_H
# df['Median_S'] = col_df.Median_S
# df['Median_V'] = col_df.Median_V

# df['h1'] = col_df.h1
# df['h2'] = col_df.h2
# df['h3'] = col_df.h3
# df['h4'] = col_df.h4

# df['s1'] = col_df.s1
# df['s2'] = col_df.s2
# df['s3'] = col_df.s3
# df['s4'] = col_df.s4

# df['v1'] = col_df.v1
# df['v2'] = col_df.v2
# df['v3'] = col_df.v3
# df['v4'] = col_df.v4
# df.head()

In [28]:
df = df.query(" (Устьица == 'гаплоид' | Устьица == 'диплоид') | (Хромосомы == 'гаплоид'| Хромосомы == 'диплоид')")
df = df.reset_index(drop=True)

In [29]:
# df.query("Name == '446-2-1'")

In [30]:
# df.iloc[275:285]

In [31]:
for i in df.index:
    if df.loc[i,'Хромосомы'] == 'гаплоид?':
        #print(i)
        df.loc[i,'Хромосомы'] = 'гаплоид'
    if df.loc[i,'Хромосомы'] == 'диплоид?':
        #print(i)
        df.loc[i,'Хромосомы'] = 'диплоид'
    if df.loc[i,'Хромосомы'] != 'гаплоид' and df.loc[i,'Хромосомы'] != 'диплоид':
        df.loc[i,'Хромосомы'] = df.loc[i,'Устьица']

In [32]:
# df.iloc[275:285]

In [33]:
df.head()

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Устьица,Хромосомы
0,453-4-1,1081.193001,8,39,8.0,18.5,40,528,941,92739,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,гаплоид,гаплоид
1,454-26-4,1358.567676,5,39,7.0,15.5,54,720,1146,117758,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,гаплоид,гаплоид
2,454-19,1438.165656,9,44,11.0,19.5,121,1159,582,91148,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,диплоид,диплоид
3,449-1-3,1380.912914,12,37,5.0,17.5,59,938,1078,87804,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,диплоид,диплоид
4,991-3-4,1222.470129,3,15,3.0,10.0,36,523,1148,50784,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,диплоид,диплоид


In [34]:
df = df.drop(columns = ['Устьица'])

In [35]:
df = df.query("Хромосомы == 'гаплоид' or Хромосомы == 'диплоид'")
df = pd.get_dummies(df,columns = ['Хромосомы'])
df.head()

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Хромосомы_гаплоид,Хромосомы_диплоид
0,453-4-1,1081.193001,8,39,8.0,18.5,40,528,941,92739,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0
1,454-26-4,1358.567676,5,39,7.0,15.5,54,720,1146,117758,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0
2,454-19,1438.165656,9,44,11.0,19.5,121,1159,582,91148,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1
3,449-1-3,1380.912914,12,37,5.0,17.5,59,938,1078,87804,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1
4,991-3-4,1222.470129,3,15,3.0,10.0,36,523,1148,50784,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1


In [36]:
# df.shape

In [37]:
def direct_name(path):
    names = path.split('/')
    direct_path = ''
    for p in names:
        if p == names[-1]:
            break
        direct_path += p + '/'
    return direct_path

In [38]:
# df.query(" Path =='/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_05 Салина гаплоиды/454-19-1.jpg'")

In [39]:
def subset_for_str(s,L):
    for l in L:
        if l == s:
            #print(l)
            return 1
    return 0

In [40]:
def first_time_point(df):
    f_df = pd.DataFrame()
    for i in range(df.shape[0]):
        if subset_for_str(direct_name(df.iloc[i].Path),first) == 1:
            f_df = f_df.append(df.iloc[i],ignore_index=True)
    f_df.reset_index(drop = True)
    return f_df

In [41]:
def first_time_point_2(df):
    f_df = pd.DataFrame()
    for i in range(df.shape[0]):
        if subset_for_str(direct_name(df.iloc[i].Path),first_2) == 1:
            f_df = f_df.append(df.iloc[i],ignore_index=True)
    f_df.reset_index(drop = True)
    return f_df

In [42]:
def second_time_point(df):
    s_df = pd.DataFrame()
    for i in range(df.shape[0]):
        if subset_for_str(direct_name(df.iloc[i].Path),last) == 1:
            s_df = s_df.append(df.iloc[i],ignore_index=True)
    s_df.reset_index(drop = True)
    return s_df

In [43]:
def second_time_point_2(df):
    s_df = pd.DataFrame()
    for i in range(df.shape[0]):
        if subset_for_str(direct_name(df.iloc[i].Path),last_2) == 1:
            s_df = s_df.append(df.iloc[i],ignore_index=True)
    s_df.reset_index(drop = True)
    return s_df

In [44]:
def name_6_folder(path):
    names = path.split('/')
    for name in names:
        if name == '6':
            return True
    return False

In [45]:
def name_5_folder(path):
    names = path.split('/')
    for name in names:
        if name == '5':
            return True
    return False

In [46]:
def train_2(name):
    for n in train:
        if name == n:
            return True
    return False

In [47]:
def test_2(name):
    for n in test:
        if name == n:
            return True
    return False

In [48]:
def val_2(name):
    for n in val:
        if name == n:
            return True
    return False

In [49]:
def first_2(path):
    names = path.split('/')
    for name in names:
        if name == '5':
            return True
    return False

In [50]:
def name_3_folder(path):
    names = path.split('/')
    for name in names:
        if name == '3':
            return True
    return False

In [51]:
def name_4_folder(path):
    names = path.split('/')
    for name in names:
        if name == '4':
            return True
    return False

In [52]:
def name_2_folder(path):
    names = path.split('/')
    for name in names:
        if name == '2':
            return True
    return False

In [53]:
def same_sample_name(sample_name):
    names = sample_name.split('-')
    if len(names) > 1:
        return names[0] + '-' + names[1]
    return sample_name

In [54]:
name_3_folder('/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_05 Салина гаплоиды/454-19-1.jpg')

True

In [55]:
same_sample_name('447-4-243-5')

'447-4'

In [56]:
df = df.assign(test = pd.Series((map(name_5_folder,df.Path))))
df = df.query("test != True")
df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Хромосомы_гаплоид,Хромосомы_диплоид,test
0,453-4-1,1081.193001,8,39,8.0,18.5,40,528,941,92739,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0,False
1,454-26-4,1358.567676,5,39,7.0,15.5,54,720,1146,117758,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0,False
2,454-19,1438.165656,9,44,11.0,19.5,121,1159,582,91148,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
3,449-1-3,1380.912914,12,37,5.0,17.5,59,938,1078,87804,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
4,991-3-4,1222.470129,3,15,3.0,10.0,36,523,1148,50784,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1004,454-67,628.445743,5,26,4.0,13.0,29,290,459,23451,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
1005,454-50-1,633.575685,3,20,3.0,9.0,20,341,592,23409,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
1006,454-59-3,661.097546,2,9,3.0,8.5,36,455,579,22880,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0,False
1007,454-52-2,553.730014,0,16,5.0,7.5,15,288,483,27706,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False


In [57]:
for stick in img_with_stick:
    df = df.query("Name != @stick")
df.shape

(701, 14)

In [58]:
df = df.drop(columns = ['test'])
df = df.reset_index(drop=True)
df.shape

(701, 13)

In [59]:
df = df.assign(test = pd.Series((map(name_2_folder,df.Path))))
df = df.query("test != True")
df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Хромосомы_гаплоид,Хромосомы_диплоид,test
0,453-4-1,1081.193001,8,39,8.0,18.5,40,528,941,92739,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0,False
1,454-26-4,1358.567676,5,39,7.0,15.5,54,720,1146,117758,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0,False
2,449-1-3,1380.912914,12,37,5.0,17.5,59,938,1078,87804,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
3,991-3-4,1222.470129,3,15,3.0,10.0,36,523,1148,50784,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
4,454-21-2,1277.834703,1,5,1.0,6.5,15,735,1079,18681,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,454-60-4,560.717821,0,20,6.0,11.0,26,380,494,34426,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0,False
697,454-49-1,558.149278,2,14,3.0,8.0,30,511,552,17237,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
698,454-50-1,633.575685,3,20,3.0,9.0,20,341,592,23409,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1,False
699,454-59-3,661.097546,2,9,3.0,8.5,36,455,579,22880,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0,False


In [60]:
df = df.drop(columns = ['test'])
df = df.reset_index(drop=True)
df.shape

(701, 13)

In [61]:
df.Name = pd.Series(map(same_sample_name,df.Name))
df.shape

(701, 13)

In [62]:
df = df.query("Name != '991-9' ")

In [63]:
df.head()

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Хромосомы_гаплоид,Хромосомы_диплоид
0,453-4,1081.193001,8,39,8.0,18.5,40,528,941,92739,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0
1,454-26,1358.567676,5,39,7.0,15.5,54,720,1146,117758,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0
2,449-1,1380.912914,12,37,5.0,17.5,59,938,1078,87804,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1
3,991-3,1222.470129,3,15,3.0,10.0,36,523,1148,50784,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1
4,454-21,1277.834703,1,5,1.0,6.5,15,735,1079,18681,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1


In [64]:
len(set(df.Name))

89

In [65]:
# for bl in black_list:
#     df = df.query("Name != @bl")
#     #print(bl)
# df.shape

In [66]:
# len(set(df.Name))

In [67]:
s_df = second_time_point(df)
s_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Хромосомы_гаплоид,Хромосомы_диплоид
0,453-4,1081.193001,8.0,39.0,8.0,18.5,40.0,528.0,941.0,92739.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0
1,454-26,1358.567676,5.0,39.0,7.0,15.5,54.0,720.0,1146.0,117758.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0
2,449-1,1380.912914,12.0,37.0,5.0,17.5,59.0,938.0,1078.0,87804.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
3,991-3,1222.470129,3.0,15.0,3.0,10.0,36.0,523.0,1148.0,50784.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
4,454-21,1277.834703,1.0,5.0,1.0,6.5,15.0,735.0,1079.0,18681.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,454-60,1285.938167,4.0,27.0,4.0,14.5,51.0,636.0,1145.0,82702.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0
336,454-49,1427.060100,1.0,12.0,2.0,7.5,22.0,813.0,1188.0,41672.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
337,454-50,1111.541197,1.0,15.0,4.0,8.5,19.0,361.0,1020.0,57481.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
338,454-59,610.333044,1.0,11.0,3.0,12.0,37.0,299.0,554.0,27632.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0


In [68]:
s_df_2 = second_time_point_2(df_2)
s_df_2

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,L_25
0,IMG_8392,1805.317026,5.0,23.0,7.0,14.5,32.0,991.0,1278.0,132696.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
1,IMG_8382,2218.848989,15.0,36.0,8.0,18.0,45.0,1008.0,1457.0,169228.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
2,IMG_8394,2829.034847,15.0,43.0,10.0,18.5,50.0,1672.0,1720.0,285048.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
3,IMG_8385,2098.116882,11.0,39.0,7.0,15.0,30.0,1070.0,1639.0,184287.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
4,IMG_8406,2072.226622,14.0,23.0,5.0,16.0,48.0,827.0,1385.0,117445.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
5,IMG_8388,1989.359523,12.0,30.0,9.0,19.0,67.0,1123.0,1670.0,193042.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
6,IMG_8409,2014.435642,8.0,39.0,9.0,16.5,47.0,1351.0,1814.0,231568.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
7,IMG_8403,2179.626550,9.0,41.0,7.0,18.0,52.0,846.0,1762.0,189668.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
8,IMG_8404,1755.165656,5.0,16.0,4.0,13.0,51.0,612.0,1275.0,83365.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
9,IMG_8398,1416.788889,2.0,8.0,3.0,10.0,32.0,479.0,1362.0,52668.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0


In [69]:
f_df = first_time_point(df)
#f_df.drop([2],axis = 0)
f_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Хромосомы_гаплоид,Хромосомы_диплоид
0,454-26,928.766594,6.0,27.0,4.0,14.5,32.0,608.0,717.0,60083.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0
1,454-21,626.842712,0.0,5.0,2.0,5.0,14.0,211.0,518.0,11597.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
2,453-4,601.925974,3.0,29.0,8.0,15.0,46.0,431.0,550.0,49840.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0
3,451-5,657.404112,4.0,19.0,4.0,16.0,71.0,507.0,615.0,35198.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
4,449-1,738.362482,4.0,25.0,5.0,13.5,31.0,559.0,727.0,57972.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,454-60,560.717821,0.0,20.0,6.0,11.0,26.0,380.0,494.0,34426.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0
349,454-49,558.149278,2.0,14.0,3.0,8.0,30.0,511.0,552.0,17237.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
350,454-50,633.575685,3.0,20.0,3.0,9.0,20.0,341.0,592.0,23409.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
351,454-59,661.097546,2.0,9.0,3.0,8.5,36.0,455.0,579.0,22880.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0


In [148]:
f_df_2 = first_time_point_2(df_2)
#f_df.drop([2],axis = 0)
f_df_2

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,L_25
0,IMG_8319,1096.825469,2.0,6.0,3.0,8.5,28.0,519.0,801.0,40563.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
1,IMG_8346,1168.886435,5.0,10.0,5.0,12.0,38.0,720.0,637.0,38280.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
2,IMG_8345,933.494516,3.0,17.0,3.0,12.0,42.0,512.0,755.0,45812.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
3,IMG_8334,1294.366666,4.0,19.0,5.0,13.5,87.0,812.0,653.0,40995.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
4,IMG_8322,1108.915872,3.0,14.0,5.0,15.5,95.0,641.0,664.0,46825.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
5,IMG_8340,763.700577,5.0,10.0,4.0,10.0,66.0,336.0,666.0,33616.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
6,IMG_8331,1523.912914,3.0,7.0,3.0,9.0,87.0,820.0,626.0,34512.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
7,IMG_8342,2144.971788,3.0,17.0,4.0,16.0,39.0,1028.0,1002.0,63770.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
8,IMG_8336,1902.827561,2.0,9.0,3.0,16.0,42.0,737.0,1175.0,50933.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
9,IMG_8344,1627.959595,5.0,10.0,4.0,8.0,39.0,891.0,927.0,40013.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0


In [72]:
two_names = list(set(f_df.Name) & set(s_df.Name))
len(two_names)

84

In [73]:
df = df.assign(test = pd.Series((map(lambda n:subset_for_str(n,two_names),df.Name))))
#df = df.query("test == 1")
df = df.query("test != 0")
df = df.drop('test',axis = 1)
df = df.reset_index(drop = True)
df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Хромосомы_гаплоид,Хромосомы_диплоид
0,453-4,1081.193001,8,39,8.0,18.5,40,528,941,92739,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0
1,454-26,1358.567676,5,39,7.0,15.5,54,720,1146,117758,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0
2,449-1,1380.912914,12,37,5.0,17.5,59,938,1078,87804,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1
3,991-3,1222.470129,3,15,3.0,10.0,36,523,1148,50784,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1
4,454-21,1277.834703,1,5,1.0,6.5,15,735,1079,18681,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,454-60,560.717821,0,20,6.0,11.0,26,380,494,34426,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0
668,454-49,558.149278,2,14,3.0,8.0,30,511,552,17237,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1
669,454-50,633.575685,3,20,3.0,9.0,20,341,592,23409,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0,1
670,454-59,661.097546,2,9,3.0,8.5,36,455,579,22880,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1,0


In [ ]:
# list(df.query("Name == '454-19'").Path)

In [ ]:
list(set(df_2.Name) -set(s_df_2.Name))

In [74]:
f_df = f_df.assign(test = pd.Series((map(lambda n:subset_for_str(n,two_names),f_df.Name))))
#df = df.query("test == 1")
f_df = f_df.query("test != 0")
f_df.drop('test',axis = 1)
f_df = f_df.reset_index(drop = True)
f_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Хромосомы_гаплоид,Хромосомы_диплоид,test
0,454-26,928.766594,6.0,27.0,4.0,14.5,32.0,608.0,717.0,60083.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0,1
1,454-21,626.842712,0.0,5.0,2.0,5.0,14.0,211.0,518.0,11597.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0,1
2,453-4,601.925974,3.0,29.0,8.0,15.0,46.0,431.0,550.0,49840.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0,1
3,451-5,657.404112,4.0,19.0,4.0,16.0,71.0,507.0,615.0,35198.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0,1
4,449-1,738.362482,4.0,25.0,5.0,13.5,31.0,559.0,727.0,57972.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,454-60,560.717821,0.0,20.0,6.0,11.0,26.0,380.0,494.0,34426.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0,1
332,454-49,558.149278,2.0,14.0,3.0,8.0,30.0,511.0,552.0,17237.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0,1
333,454-50,633.575685,3.0,20.0,3.0,9.0,20.0,341.0,592.0,23409.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0,1
334,454-59,661.097546,2.0,9.0,3.0,8.5,36.0,455.0,579.0,22880.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0,1


In [75]:
s_df = s_df.assign(test = pd.Series((map(lambda n:subset_for_str(n,two_names),s_df.Name))))
#df = df.query("test == 1")
s_df = s_df.query("test != 0")
s_df = s_df.drop('test',axis = 1)
s_df = s_df.reset_index(drop = True)
s_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,Хромосомы_гаплоид,Хромосомы_диплоид
0,453-4,1081.193001,8.0,39.0,8.0,18.5,40.0,528.0,941.0,92739.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0
1,454-26,1358.567676,5.0,39.0,7.0,15.5,54.0,720.0,1146.0,117758.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0
2,449-1,1380.912914,12.0,37.0,5.0,17.5,59.0,938.0,1078.0,87804.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
3,991-3,1222.470129,3.0,15.0,3.0,10.0,36.0,523.0,1148.0,50784.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
4,454-21,1277.834703,1.0,5.0,1.0,6.5,15.0,735.0,1079.0,18681.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,454-60,1285.938167,4.0,27.0,4.0,14.5,51.0,636.0,1145.0,82702.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0
332,454-49,1427.060100,1.0,12.0,2.0,7.5,22.0,813.0,1188.0,41672.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
333,454-50,1111.541197,1.0,15.0,4.0,8.5,19.0,361.0,1020.0,57481.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,0.0,1.0
334,454-59,610.333044,1.0,11.0,3.0,12.0,37.0,299.0,554.0,27632.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...,1.0,0.0


In [ ]:
# len(set(f_df.Name))

In [ ]:
# len(set(s_df.Name))

In [76]:
mean_df = df.groupby('Name',as_index = False).aggregate({'Lenght':'mean','Peregib':'mean','Leaf_and_Root':'mean',
                                            'line_kust':'mean','line_kust_value':'mean','max_kust':'mean',
                                            'width_rectangle':'mean', 'height_rectangle':'mean', 'bio_mass':'mean',
                                            'Хромосомы_гаплоид':'max','Path':'max'})
mean_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Хромосомы_гаплоид,Path
0,446-2,1133.250658,10.250,34.000,6.1875,18.5625,58.500,783.750,894.750,89194.875,1,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
1,446-3,766.869255,2.500,21.250,5.6250,11.1875,36.375,547.375,497.500,32975.875,0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
2,446-4,576.850811,1.875,15.000,2.7500,8.0000,34.625,424.625,464.250,16784.125,0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
3,446-5,414.281268,2.750,26.875,5.0000,10.4375,46.375,472.000,204.250,8061.625,0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
4,446-6,466.034839,1.500,12.000,3.5000,6.5625,19.625,364.750,263.125,8417.750,1,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
...,...,...,...,...,...,...,...,...,...,...,...,...
79,991-12,586.448791,1.875,28.375,3.2500,7.1875,22.500,336.750,497.000,16193.375,1,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
80,991-13,1110.169922,4.375,37.125,6.5000,16.8750,54.500,734.375,903.500,79107.625,0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
81,991-2,1073.433378,3.125,31.375,6.5000,14.0625,35.500,671.875,847.000,80103.000,0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
82,991-3,1133.132124,3.250,19.500,3.0000,11.5625,33.750,645.000,930.625,41237.000,0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...


In [ ]:
# mean_df.index

In [77]:
mean_f_df = f_df.groupby('Name',as_index = False).aggregate({'Lenght':'mean','Peregib':'mean','Leaf_and_Root':'mean',
                                            'line_kust':'mean','line_kust_value':'mean','max_kust':'mean',
                                            'width_rectangle':'mean', 'height_rectangle':'mean', 'bio_mass':'mean',
                                            'Хромосомы_гаплоид':'max','Path':'max'})
mean_f_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Хромосомы_гаплоид,Path
0,446-2,762.084614,9.00,26.75,6.375,16.250,45.00,480.25,582.25,56827.00,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
1,446-3,583.613005,2.00,16.25,3.750,9.875,27.25,350.25,413.75,20622.50,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
2,446-4,566.813492,0.75,18.50,2.000,5.625,17.00,321.25,530.75,11419.50,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
3,446-5,386.211724,2.00,35.50,4.875,9.500,50.50,414.75,255.25,9314.75,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
4,446-6,190.540368,0.25,9.50,2.000,4.625,14.75,176.75,162.00,2220.75,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
...,...,...,...,...,...,...,...,...,...,...,...,...
79,991-12,498.442695,1.50,41.50,3.500,7.750,28.50,300.25,388.50,12586.00,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
80,991-13,864.693957,3.00,39.00,5.500,15.125,49.00,584.00,806.50,49339.50,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
81,991-2,724.360479,2.25,28.25,5.750,11.375,28.50,415.25,634.75,50705.50,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
82,991-3,818.539412,3.50,19.00,2.750,12.250,34.50,414.00,723.75,27963.00,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...


In [ ]:
# mean_f_df.shape

In [78]:
mean_s_df = s_df.groupby('Name',as_index = False).aggregate({'Lenght':'mean','Peregib':'mean','Leaf_and_Root':'mean',
                                            'line_kust':'mean','line_kust_value':'mean','max_kust':'mean',
                                            'width_rectangle':'mean', 'height_rectangle':'mean', 'bio_mass':'mean',
                                            'Хромосомы_гаплоид':'max','Path':'max'})
mean_s_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Хромосомы_гаплоид,Path
0,446-2,1504.416702,11.50,41.25,6.000,20.875,72.00,1087.25,1207.25,121562.75,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
1,446-3,950.125504,3.00,26.25,7.500,12.500,45.50,744.50,581.25,45329.25,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
2,446-4,586.888131,3.00,11.50,3.500,10.375,52.25,528.00,397.75,22148.75,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
3,446-5,442.350811,3.50,18.25,5.125,11.375,42.25,529.25,153.25,6808.50,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
4,446-6,741.529310,2.75,14.50,5.000,8.500,24.50,552.75,364.25,14614.75,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
...,...,...,...,...,...,...,...,...,...,...,...,...
79,991-12,674.454888,2.25,15.25,3.000,6.625,16.50,373.25,605.50,19800.75,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
80,991-13,1355.645886,5.75,35.25,7.500,18.625,60.00,884.75,1000.50,108875.75,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
81,991-2,1422.506276,4.00,34.50,7.250,16.750,42.50,928.50,1059.25,109500.50,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...
82,991-3,1447.724837,3.00,20.00,3.250,10.875,33.00,876.00,1137.50,54511.00,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды...


In [ ]:
# mean_s_df.shape

In [79]:
def get_dif_df(f_df,s_df):
    l = []
    for n in f_df.index:
        for n2 in s_df.index:
            if str(n) == str(n2):
                l1 = []
                #print(s_df.loc[n2].Bio_Mass - f_df.loc[n].Bio_Mass)
                l1.append(s_df.loc[n2].Name)
            
                l1.append(s_df.loc[n2].Lenght - f_df.loc[n].Lenght)
                l1.append(s_df.loc[n2].Peregib - f_df.loc[n].Peregib)
                l1.append(s_df.loc[n2].Leaf_and_Root - f_df.loc[n].Leaf_and_Root)
            
                l1.append(s_df.loc[n2].line_kust - f_df.loc[n].line_kust)
                l1.append(s_df.loc[n2].line_kust_value - f_df.loc[n].line_kust_value)
                l1.append(s_df.loc[n2].max_kust - f_df.loc[n].max_kust)
                l1.append(s_df.loc[n2].width_rectangle - f_df.loc[n].width_rectangle)
            
                l1.append(s_df.loc[n2].height_rectangle - f_df.loc[n].height_rectangle)
                l1.append(s_df.loc[n2].bio_mass - f_df.loc[n].bio_mass)
               
                l1.append(s_df.loc[n2].Хромосомы_гаплоид)
                l1.append(s_df.loc[n2].Path)
            
                l.append(l1)

    dif_df =  pd.DataFrame(l,columns = ['Name','Lenght','Peregib','Leaf_and_Root', 
                                        'line_kust','line_kust_value','max_kust','width_rectangle',
                                        'height_rectangle','bio_mass','Хромосомы_гаплоид', 'Path',])          
    return dif_df        

In [ ]:
dif_df_2 = get_dif_df(f_df_2,s_df_2)
dif_df_2

In [ ]:
mean_dif_df = get_dif_df(mean_f_df,mean_s_df)
mean_dif_df

In [ ]:
list(mean_dif_df.Path)

In [ ]:
sns.histplot(data = mean_df['Bio_Mass'].apply(math.sqrt), kde = True)

In [ ]:
k2, p = stats.normaltest(mean_df['Bio_Mass'].apply(math.sqrt))
p

In [ ]:
sns.histplot(data = mean_df['widht'], kde = True)

In [ ]:
k2, p = stats.normaltest(mean_df['widht'])
p

In [ ]:
sns.histplot(data = mean_df['height'], kde = True)

In [ ]:
k2, p = stats.normaltest(mean_df['height'])
p

In [ ]:
sns.histplot(data = mean_df['Median_H'], kde = True)

In [ ]:
k2, p = stats.normaltest(mean_s_df['Median_H'])
p

In [ ]:
sns.histplot(data = mean_df['Median_S'], kde = True)

In [ ]:
k2, p = stats.normaltest(mean_df['Median_S'])
p

In [ ]:
sns.histplot(data = mean_df['Median_V'], kde = True)

In [ ]:
k2, p = stats.normaltest(mean_df['Median_S'])
p

In [81]:
feature = 'height_rectangle'
stat_df = mean_df

In [ ]:
# stat_f_df = stat_df.assign(test = pd.Series((map(name_4_folder,stat_df.Path))))
# stat_f_df = stat_f_df.query("test == True ")
# stat_f_df = stat_f_df.drop(columns = ['test'])
# stat_f_df = stat_f_df.drop(columns = ['Path'])

In [ ]:
# stat_s_df = stat_df.assign(test = pd.Series((map(name_3_folder,stat_df.Path))))
# stat_s_df = stat_s_df.query("test == True ")
# stat_s_df = stat_s_df.drop(columns = ['test'])
# stat_s_df = stat_s_df.drop(columns = ['Path'])

In [82]:
stat_f_df = stat_df.query("Хромосомы_гаплоид == 1")
stat_s_df = stat_df.query("Хромосомы_гаплоид == 0  ")

In [83]:
stat_f_df = stat_f_df.assign(mas = np.sqrt(stat_f_df[feature]))
stat_s_df = stat_s_df.assign(mas = np.sqrt(stat_s_df[feature]))
#stat_s_df

In [ ]:
sns.histplot(data = stat_f_df[feature], kde = True, color = 'red')
sns.histplot(data = stat_s_df[feature], kde = True)

In [ ]:
res = stats.probplot(stat_f_df[feature], plot=plt)

In [ ]:
res = stats.probplot(stat_s_df[feature], plot=plt)

In [ ]:
k2, p = stats.normaltest(stat_f_df[feature])
p

In [ ]:
k2, p = stats.normaltest(stat_s_df[feature])
p

In [ ]:
stat, p = bartlett(stat_f_df[feature], stat_s_df[feature])
p

In [ ]:
stats.ttest_ind(stat_f_df[feature], stat_s_df[feature])

In [ ]:
U1, p = mannwhitneyu(stat_f_df[feature], stat_s_df[feature])
p

In [ ]:
stats.kruskal(stat_f_df[feature],stat_s_df[feature])

In [ ]:
stat_f_df.query("height_rectangle < -200")

In [ ]:
stolb_names = ['Lenght','Peregib','Leaf_and_Root','line_kust','line_kust_value',
               'width_rectangle','height_rectangle','bio_mass']

In [ ]:
def norm(df):
    for st in stolb_names:
        m_1 = df[st].mean()
        st_1 = df[st].std()
    
        for i in df.index:
            df.loc[i,st] = (df.loc[i,st] - m_1)/st_1

In [ ]:
norm(df_2)
df_2

In [ ]:
norm(f_df_2)
norm(s_df_2)
norm(dif_df_2)

In [ ]:
norm(mean_df)
norm(mean_f_df)
norm(mean_s_df)
norm(mean_dif_df)

In [ ]:
mean_df

In [174]:
forest_df = f_df_2
forest_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,max_kust,width_rectangle,height_rectangle,bio_mass,Path,L_25
0,IMG_8319,1096.825469,2.0,6.0,3.0,8.5,28.0,519.0,801.0,40563.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
1,IMG_8346,1168.886435,5.0,10.0,5.0,12.0,38.0,720.0,637.0,38280.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
2,IMG_8345,933.494516,3.0,17.0,3.0,12.0,42.0,512.0,755.0,45812.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
3,IMG_8334,1294.366666,4.0,19.0,5.0,13.5,87.0,812.0,653.0,40995.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
4,IMG_8322,1108.915872,3.0,14.0,5.0,15.5,95.0,641.0,664.0,46825.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
5,IMG_8340,763.700577,5.0,10.0,4.0,10.0,66.0,336.0,666.0,33616.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
6,IMG_8331,1523.912914,3.0,7.0,3.0,9.0,87.0,820.0,626.0,34512.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
7,IMG_8342,2144.971788,3.0,17.0,4.0,16.0,39.0,1028.0,1002.0,63770.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
8,IMG_8336,1902.827561,2.0,9.0,3.0,16.0,42.0,737.0,1175.0,50933.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
9,IMG_8344,1627.959595,5.0,10.0,4.0,8.0,39.0,891.0,927.0,40013.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0


In [175]:
train_df = forest_df.assign(test = pd.Series((map(test_2,forest_df.Name))))
train_df = train_df.query("test != True ")
train_df = train_df.drop(columns = ['test'])
train_df = train_df.groupby('Name',as_index = False).aggregate({'Lenght':'mean','Peregib':'mean','Leaf_and_Root':'mean',
                                            'line_kust':'mean','line_kust_value':'mean',
                                            'width_rectangle':'mean', 'height_rectangle':'mean', 'bio_mass':'mean',
                                            'L_25':'max','Path':'max'})
train_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,width_rectangle,height_rectangle,bio_mass,L_25,Path
0,IMG_8324,1288.790981,4.0,18.0,4.0,9.0,554.0,880.0,49737.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
1,IMG_8325,1320.420490,2.0,10.0,6.0,11.5,964.0,608.0,53121.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
2,IMG_8326,1029.482323,3.0,9.0,3.0,9.5,630.0,764.0,28859.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
3,IMG_8327,1560.775829,3.0,8.0,3.0,10.0,716.0,1082.0,46473.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
4,IMG_8328,1171.702669,2.0,6.0,2.0,10.0,876.0,1001.0,35896.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
5,IMG_8329,1177.643794,2.0,30.0,4.0,8.5,814.0,881.0,46628.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
6,IMG_8330,1189.964646,6.0,12.0,4.0,10.0,614.0,718.0,26988.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
7,IMG_8331,1523.912914,3.0,7.0,3.0,9.0,820.0,626.0,34512.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
8,IMG_8332,1485.052957,5.0,14.0,4.0,11.5,918.0,842.0,45872.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
9,IMG_8333,1136.261110,4.0,11.0,5.0,14.5,609.0,839.0,52533.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...


In [176]:
test_df = forest_df.assign(test = pd.Series((map(test_2,forest_df.Name))))
test_df = test_df.query("test == True ")
test_df = test_df.drop(columns = ['test'])

test_df = test_df.groupby('Name',as_index = False).aggregate({'Lenght':'mean','Peregib':'mean','Leaf_and_Root':'mean',
                                            'line_kust':'mean','line_kust_value':'mean',
                                            'width_rectangle':'mean', 'height_rectangle':'mean', 'bio_mass':'mean',
                                            'L_25':'max','Path':'max'})
test_df

,Name,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,width_rectangle,height_rectangle,bio_mass,L_25,Path
0,IMG_8319,1096.825469,2.0,6.0,3.0,8.5,519.0,801.0,40563.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
1,IMG_8320,1546.555483,3.0,12.0,3.0,8.5,466.0,797.0,31300.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
2,IMG_8321,1495.506709,2.0,7.0,4.0,9.5,738.0,897.0,52431.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
3,IMG_8322,1108.915872,3.0,14.0,5.0,15.5,641.0,664.0,46825.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
4,IMG_8323,1469.319985,7.0,14.0,4.0,14.5,742.0,848.0,51814.0,1.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
5,IMG_8351,888.776695,0.0,10.0,3.0,3.5,278.0,851.0,29351.0,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
6,IMG_8352,840.806133,0.0,7.0,2.0,3.0,187.0,800.0,18041.0,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
7,IMG_8353,873.404112,0.0,3.0,3.0,4.5,242.0,819.0,25014.0,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
8,IMG_8354,866.536147,1.0,6.0,2.0,4.5,535.0,749.0,24956.0,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...
9,IMG_8355,660.918831,0.0,6.0,3.0,4.5,304.0,618.0,25432.0,0.0,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...


In [ ]:
sns.histplot(data = train_df, x = 'Хромосомы_гаплоид', color = 'red')

In [ ]:
len(list(train_df.query("Хромосомы_гаплоид == 1").index))

In [ ]:
len(list(train_df.query("Хромосомы_гаплоид == 0").index))

In [ ]:
sns.histplot(data = test_df, x = 'Хромосомы_гаплоид')

In [ ]:
len(list(test_df.query("Хромосомы_гаплоид == 1").index))

In [ ]:
len(list(test_df.query("Хромосомы_гаплоид == 0").index))

In [177]:
X_train = train_df.drop(['L_25','Name'],axis = 1)
X_train = X_train.drop(columns = ['Path'])
X_train.head()

,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,width_rectangle,height_rectangle,bio_mass
0,1288.790981,4.0,18.0,4.0,9.0,554.0,880.0,49737.0
1,1320.420490,2.0,10.0,6.0,11.5,964.0,608.0,53121.0
2,1029.482323,3.0,9.0,3.0,9.5,630.0,764.0,28859.0
3,1560.775829,3.0,8.0,3.0,10.0,716.0,1082.0,46473.0
4,1171.702669,2.0,6.0,2.0,10.0,876.0,1001.0,35896.0


In [178]:
X_test = test_df.drop(['L_25','Name'],axis = 1)
X_test = X_test.drop(columns = ['Path'])
X_test.head()

,Lenght,Peregib,Leaf_and_Root,line_kust,line_kust_value,width_rectangle,height_rectangle,bio_mass
0,1096.825469,2.0,6.0,3.0,8.5,519.0,801.0,40563.0
1,1546.555483,3.0,12.0,3.0,8.5,466.0,797.0,31300.0
2,1495.506709,2.0,7.0,4.0,9.5,738.0,897.0,52431.0
3,1108.915872,3.0,14.0,5.0,15.5,641.0,664.0,46825.0
4,1469.319985,7.0,14.0,4.0,14.5,742.0,848.0,51814.0


In [179]:
y_test = test_df[['L_25']]
y_test.head()

,L_25
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [180]:
y_train = train_df[['L_25']]
y_train.head()

,L_25
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


In [ ]:
clf =RandomForestClassifier()
parametrs = {'n_estimators':range(2,12),'max_depth':range(2,4)}
grid_search_cv_clf = GridSearchCV(clf,parametrs,cv = 4,scoring = 'accuracy')

In [ ]:
grid_search_cv_clf.fit(X_train,y_train)

In [ ]:
grid_search_cv_clf.best_params_

In [ ]:
clf_forest = grid_search_cv_clf.best_estimator_
clf_forest.score(X_train,y_train)

In [ ]:
clf_forest.score(X_test,y_test)

In [ ]:
clf_forest

In [ ]:
clf_forest.feature_importances_

In [ ]:
clf_reg = LogisticRegressionCV(cv=4, random_state=0).fit(X_train,y_train)

In [ ]:
clf_reg.score(X_train,y_train)

In [ ]:
clf_reg.score(X_test,y_test)

In [ ]:
clf_reg.get_params(deep=True)

In [ ]:
clf_reg.coef_

In [96]:
def direct_name(path):
    names = path.split('/')
    direct_path = ''
    for p in names:
        if p == names[-1]:
            break
        direct_path += p + '/'
    return direct_path

In [97]:
def all_sample_name(sample_name):
    names = sample_name.split('/')[-1].split('.')[0]
    dir_name = direct_name(sample_name)
    name = names.split('-')[0] + '-'+ names.split('-')[1]
    one = dir_name + name + '-' + '1' + '.jpg'
    two = dir_name +  name + '-' + '2'  + '.jpg'
    three = dir_name + name + '-' + '3'  + '.jpg'
    four =dir_name + name + '-' + '4' + '.jpg'
    stick =dir_name + name + '.jpg'
    return one,two,three,four,stick

In [98]:
all_sample_name('/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/452-3-2.jpg')

('/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/452-3-1.jpg',
 '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/452-3-2.jpg',
 '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/452-3-3.jpg',
 '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/452-3-4.jpg',
 '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/452-3.jpg')

In [100]:
def augmented_load(img, aug):
    #img = torch.from_numpy(img).long()
    img = transforms.ToPILImage()(img)
    aug_img = aug(img)
    #aug_img = auged['image']

    aug_img = aug_img.numpy().transpose(0, 1, 2)

    return aug_img


In [101]:
img = cv2.imread('/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/452-3-2.jpg')
img = torch.cuda.FloatTensor(augmented_load(img, infer_aug))
img.unsqueeze_(0)
outputs = resnet_all(img)
_, preds = torch.max(outputs, 1)
int(preds[0])

0

In [181]:
def rescale(img,mask):
    new_img = np.zeros((1505,1505,3),dtype = np.uint8)
  #  new_img = np.zeros((2000,2000,3),dtype = np.uint8)
    w,h,x,y = rectangle(mask)
    w_2 = int(w/2)
    h_2 = int(h/2)
    x = int(x)
    y = int(y)
    
    s_img = img[y - h_2:y + h_2,x - w_2:x + w_2,:]
    new_img[0: s_img.shape[0], 0: s_img.shape[1]] = s_img
    
    return new_img

In [103]:
def sample_name(path):
    names = path.split('/')
    names = names[-1].split('.')
    return names[0] 

In [104]:
def filename_pref_png(name, pref):
    names = name.split('/')
    png_names = names[-1].split('.')
    names[-1] = png_names[0] + '.' + 'png'
    names[-1] = pref + names[-1]
    new_name = ''
    for n in names:
        if n != names[-1]:
            new_name =new_name + n + '/'
        else:
            new_name =new_name + n
    return new_name    

In [105]:
def rectangle(mask):
    new_mask = np.where(mask > 0, 1,0)
    non_zero_mask = np.nonzero(new_mask)
    h = non_zero_mask[0].max() - non_zero_mask[0].min()
    w = non_zero_mask[1].max() - non_zero_mask[1].min()
    x = set(non_zero_mask[1])
    y = set(non_zero_mask[0])
    x = sum(x)/len(x)
    y = sum(y)/len(y)
    return w,h,x,y

In [106]:
def img_seg(mask,img):
    seg_img = img.copy()
    new_mask = np.clip(mask,0,1)
    for i in range(3):
        seg_img[:,:,i] *=new_mask
    return seg_img

In [107]:
def mask_name(path):
    dir_name = direct_name(path)
    s_name = sample_name(path)
    return dir_name + filename_pref_png(s_name, 'mask_')

In [158]:
def resnet_res(resnet,name):
    chromosm = 0;
    #names = all_sample_name(name)
    #for n in names:
    img = cv2.imread(name)
    mask = cv2.imread(mask_name(name),cv2.IMREAD_GRAYSCALE)
    img_res = cv2.resize(img,(int(mask.shape[1]), int(mask.shape[0])), interpolation = cv2.INTER_AREA)
    img = rescale(img_seg(mask,img_res),mask)
    img = torch.cuda.FloatTensor(augmented_load(img, infer_aug))
    img.unsqueeze_(0).to('cuda')
    resnet.eval()
    outputs = resnet(img).to('cuda')
    _, preds = torch.max(outputs, 1)
    chromosm += int(preds[0])
#     if chromosm > 2:
#         return 0
#     else:
#         return 1
    #print(chromosm)
    return chromosm 

In [ ]:
resnet_res(resnet_all,'/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/3/2020_10_19/452-3-2.jpg')

In [160]:
def score_resnet(df,resnet,df2):
    accur = 0
    size = 0
    for i in df.index:
        for j in df2.index:
            if df.loc[i,'Name'] == df2.loc[j,'Name']:
                if df.loc[i,'Path'] == '/home/jupyter-fly_cv/Igor/Plants/data/Гаплоиды/4/2020_11_02/991-9-4.jpg':
                    continue 
                print(df.loc[i,'Path'])
                res = resnet_res(resnet,df.loc[i,'Path'])
                size += 1
                if (df.loc[i,'L_25'] == res):
                    accur+=1
    return accur/size

In [149]:
new_f = f_df_2[['Name','Path','L_25']]
new_f

,Name,Path,L_25
0,IMG_8319,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
1,IMG_8346,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
2,IMG_8345,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
3,IMG_8334,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
4,IMG_8322,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
5,IMG_8340,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
6,IMG_8331,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
7,IMG_8342,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
8,IMG_8336,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
9,IMG_8344,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0


In [152]:
new_s = s_df_2[['Name','Path','L_25']]
new_s

,Name,Path,L_25
0,IMG_8392,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
1,IMG_8382,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
2,IMG_8394,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
3,IMG_8385,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
4,IMG_8406,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
5,IMG_8388,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
6,IMG_8409,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
7,IMG_8403,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
8,IMG_8404,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
9,IMG_8398,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0


In [153]:
new_df= pd.concat([new_s,new_f])
new_df = new_df.reset_index(drop=True)
new_df

,Name,Path,L_25
0,IMG_8392,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
1,IMG_8382,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
2,IMG_8394,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
3,IMG_8385,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
4,IMG_8406,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,1.0
...,...,...,...
115,IMG_8358,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,0.0
116,IMG_8373,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,0.0
117,IMG_8375,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,0.0
118,IMG_8379,/home/jupyter-fly_cv/Igor/Plants/data/Новый пр...,0.0


In [ ]:
resnet_18_test = torch.load(path_resnet_18_second)
resnet_50_test = torch.load(path_resnet_50_second)
resnet_101_test = torch.load(path_resnet_101_second)

In [ ]:
example_df = mean_s_df

In [ ]:
test_df = example_df.assign(test = pd.Series((map(name_4_folder,example_df.Path))))
test_df = test_df.query("test == True ")
test_df = test_df.drop(columns = ['test'])

In [ ]:
len(list(test_df.query("Хромосомы_гаплоид == 1").index))

In [ ]:
score_resnet(test_df,resnet_18_test,new_df)

In [ ]:
score_resnet(test_df,resnet_50_test,new_df)

In [ ]:
score_resnet(test_df,resnet_101_test,new_df)

In [ ]:
test_df = example_df.assign(test = pd.Series((map(name_4_folder,example_df.Path))))
test_df = test_df.query("test == True ")
test_df = test_df.drop(columns = ['test'])

In [ ]:
score_resnet(test_df,resnet_18_test,new_df)

In [ ]:
score_resnet(test_df,resnet_50_test,new_df)

In [ ]:
score_resnet(test_df,resnet_101_test,new_df)

In [ ]:
test_df = example_df.assign(test = pd.Series((map(name_6_folder,example_df.Path))))
test_df = test_df.query("test == True ")
test_df = test_df.drop(columns = ['test'])

In [ ]:
score_resnet(test_df,resnet_18_test,new_df)

In [ ]:
score_resnet(test_df,resnet_50_test,new_df)

In [ ]:
score_resnet(test_df,resnet_101_test,new_df)

In [ ]:
def comb(df2,df,X,resnet,forest,reg):
    accur = 0
    forest_res = forest.predict(X)
    reg_res = reg.predict(X)
    size = 0
    for i in df.index: 
        for j in df2.index:
            if df.loc[i,'Name'] == df2.loc[j,'Name']:
                net_res = resnet_res(resnet,df.loc[i,'Path'])
                res = net_res + forest_res[i] + reg_res[i]
                size+=1
                if (res >= 2 and df.loc[i,'Хромосомы_гаплоид'] == 1):
                    accur+=1
                if (res < 2 and df.loc[i,'Хромосомы_гаплоид'] == 0):
                    accur+=1
    return accur/size

In [ ]:
clf_forest.predict(X_train)

In [ ]:
np.array(list(y_train.Хромосомы_гаплоид))-clf_forest.predict(X_train)

In [ ]:
comb(new_s,train_df,X_train,resnet_second,clf_forest,clf_reg)